# Review of Day 1

During Day 1, we introduced Julia basics, and saw that Julia has the following features:

- Familiar syntax
- Easy to make generic code ("write once, run everywhere")
- Julia is fast

Day 2 will be about Performance and Parallelism.
We will see how to profile code, some performance gotchas, and several ways to parallelize code using Julia.

# Simulations: profiling and performance
## Random walks

In this notebook, we will look at one of the simplest types of Monte Carlo numerical simulation, random walks.

In the simplest random walk, a particle starts at $0$ and jumps to the left ($-1$) or the right ($+1$) with equal probability.

The following is a simple implementation of a single random walk:

In [55]:
numsteps = 1000
pos = 0 

for j in 1:numsteps
            
    if rand() < 0.5
        step = -1
    else
        step = +1
    end
            
    pos += step 
    
end

pos

-26

The code seems to execute almost instantaneously, but we should **profile** (time) it:

In [59]:
memory = @allocated rand(1000)

8128

In [60]:
memory

8128

In [61]:
using Interact

In [63]:
exponents = 3:7
values = [10^k for k in exponents]

5-element Array{Int64,1}:
     1000
    10000
   100000
  1000000
 10000000

In [64]:
@manipulate for k in exponents
    @allocated rand(10^k)
end

Interact.Slider{Int64}(Signal{Int64}(5, nactions=0),"k",5,3:7,"d",true)

800096

You might expect exactly $8*(10^k)$, but there is a bit of overhead in a Julia array (80 bytes?)

In [56]:
@time begin
    
    numsteps = 1000
    pos = 0 
    for j in 1:numsteps

        if rand() < 0.5
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
end

  0.000186 seconds (1.98 k allocations: 46.578 KB)


In [65]:
@elapsed sin(10)

0.00406933

In [66]:
@allocated sin(10)

0

Although it's fast, it seems to be allocating memory unexpectedly. This is a **warning** of a possible **type instability**.

Let's wrap it in a function, which is good programming practice, and allows us to have `numsteps` as a paramater.
It turns out to have an additional, important effect in Julia.

In [1]:
"""Single 1D random walk from the origin.
Returns the final position after `numsteps` steps."""
function walk(numsteps=1000)  # default value of the parameter
    
    pos = 0 
    
    for j in 1:numsteps

        if rand() < 0.5   # can replace by rand(Bool)
            step = -1
        else
            step = +1
        end

        pos += step 
    end
    
    return pos
    
end

walk

In [68]:
@time walk(1)

  0.007574 seconds (2.86 k allocations: 130.777 KB)


16

In [69]:
@time walk(1000)

  0.000011 seconds (4 allocations: 160 bytes)


-12

In [70]:
@time walk(1000)

  0.000011 seconds (4 allocations: 160 bytes)


-46

In [74]:
@manipulate for k in 3:10
    log10(@elapsed walk(10^k))
end

Interact.Slider{Int64}(Signal{Int64}(6, nactions=0),"k",6,3:10,"d",true)

-2.11911269820074

In [81]:
rand(1:10)

2

In [82]:
a = true
typeof(a)

Bool

In [92]:
rand(Bool)

false

In [93]:
@which rand(Bool)

rand(T::Type) at random.jl:228

In [94]:
@edit rand(Bool)

In [96]:
ENV["EDITOR"] = "vim"

"vim"

In [ ]:
@edit rand(Bool)

Vim: Warning: Output is not to a terminal
Vim: Warning: Input is not from a terminal




























[O]pen Read-Only, (E)dit anyway, (R)ecover, (D)elete it, (Q)uit, (A)bort: 

In [95]:
ENV

Base.EnvHash with 35 entries:
  "FONTCONFIG_PATH"            => "/Applications/Julia-0.5.app/Contents/Resourc…
  "TERM_PROGRAM_VERSION"       => "3.0.12"
  "TMPDIR"                     => "/var/folders/h0/68t1xc991mq20mrkn6yy6n5h0000…
  "LOGNAME"                    => "dpsanders"
  "USER"                       => "dpsanders"
  "HOME"                       => "/Users/dpsanders"
  "PATH"                       => "/Applications/Julia-0.5.app/Contents/Resourc…
  "CLICOLOR"                   => "1"
  "DISPLAY"                    => "/private/tmp/com.apple.launchd.I4LZaH4k5G/or…
  "TERM_PROGRAM"               => "iTerm.app"
  "LANG"                       => "en_US.UTF-8"
  "TK_LIBRARY"                 => "/System/Library/Frameworks/Tk.framework/Vers…
  "TERM"                       => "xterm-256color"
  "SHELL"                      => "/bin/bash"
  "COLORFGBG"                  => "7;0"
  "SHLVL"                      => "1"
  "XPC_FLAGS"                  => "0x0"
  "ITERM_SESSION_ID"          

Messages: 
- Wrap everything in a function
- Run the function once, before timing only on the *second* run

## Interlude on vectors: collecting data

Now we run it several times to collect data:

In [2]:
walk()

-24

In [5]:
numsteps = 10000
data = [walk(numsteps) for i in 1:10]

10-element Array{Int64,1}:
  120
 -152
 -112
   16
  138
   74
  198
  -96
  -74
   58

Another possibility:

In [6]:
[ ]

0-element Array{Any,1}

In [9]:
Int[ ]   # better for performance

0-element Array{Int64,1}

In [10]:
[ 0 ]

1-element Array{Int64,1}:
 0

In [12]:
[ 0, 1.0 ]  # it promotes everything to a common type

2-element Array{Float64,1}:
 0.0
 1.0

In [14]:
v = [ 0, 1.0, "hello"]

3-element Array{Any,1}:
 0       
 1.0     
  "hello"

In [15]:
push!(v, false)

4-element Array{Any,1}:
     0       
     1.0     
      "hello"
 false       

In [16]:
v

4-element Array{Any,1}:
     0       
     1.0     
      "hello"
 false       

The "bang" (`!`) is a convention that means that the function **modifies** its (first) argument.

In [18]:
data = Int[ ]   # empty array of `Int`s

0-element Array{Int64,1}

In [20]:
x = 3

3

In [21]:
show(x)

3

In [22]:
@show x

x = 3


3

In [23]:
data = Int[ ]   # empty array of `Int`s

numwalkers = 10

for i in 1:numwalkers
    pos = walk()
    push!(data, pos)
    @show data
end

data

data = [2]
data = [2,78]
data = [2,78,-26]
data = [2,78,-26,14]
data = [2,78,-26,14,-8]
data = [2,78,-26,14,-8,40]
data = [2,78,-26,14,-8,40,54]
data = [2,78,-26,14,-8,40,54,14]
data = [2,78,-26,14,-8,40,54,14,-10]
data = [2,78,-26,14,-8,40,54,14,-10,-58]


10-element Array{Int64,1}:
   2
  78
 -26
  14
  -8
  40
  54
  14
 -10
 -58

Alternative: pre-allocate the array

In [24]:
numwalkers = 10

data = zeros(numwalkers)

10-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [25]:
data = zeros(Int, numwalkers)

10-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [26]:
data = Vector{Int}(numwalkers)  # fills with random garbage; cf. malloc

10-element Array{Int64,1}:
    4726803984
            13
    4647731232
    4728607248
             0
    4726933952
    4726934032
    4726804016
    4726934112
 1099528470529

In [27]:
data = Vector{Int}(numwalkers)  # fills with random garbage; cf. malloc

for i in 1:numwalkers
    pos = walk()
    data[i] = pos  # this does **not** automatically grow the array (as it would in Matlab)
    @show data
end

data

data = [44,13,4647731232,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,4647731232,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,4728608784,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,0,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,4727617776,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,4727617856,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,4727334832,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,4727617936,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,-36,1099528470529]
data = [44,-12,-2,-62,34,-24,42,14,-36,22]


10-element Array{Int64,1}:
  44
 -12
  -2
 -62
  34
 -24
  42
  14
 -36
  22

In [28]:
data = Int[] # fills with random garbage; cf. malloc

for i in 1:numwalkers
    pos = walk()
    data = [data; pos]  # common Matlabism
    @show data
end

data

data = [44]
data = [44,30]
data = [44,30,-30]
data = [44,30,-30,48]
data = [44,30,-30,48,24]
data = [44,30,-30,48,24,-18]
data = [44,30,-30,48,24,-18,56]
data = [44,30,-30,48,24,-18,56,18]
data = [44,30,-30,48,24,-18,56,18,4]
data = [44,30,-30,48,24,-18,56,18,4,4]


10-element Array{Int64,1}:
  44
  30
 -30
  48
  24
 -18
  56
  18
   4
   4

In [29]:
function method1(N)  # Matlabism
    data = Int[] 

    for i in 1:N
        pos = walk()
        data = [data; pos]  
    end
    
    data  # will return data
end

method1 (generic function with 1 method)

In [30]:
function method2(N)  # push!
    data = Int[ ]   

    for i in 1:N
        pos = walk()
        push!(data, pos)
    end

    data
end

method2 (generic function with 1 method)

In [31]:
function method3(N)  # preallocate
    data = zeros(Int, N)  

    for i in 1:N
        pos = walk()
        data[i] = pos
    end

    data
end

method3 (generic function with 1 method)

First thing to do when profiling in Julia: run the functions once each

In [32]:
method1(1)
method2(1)
method3(1)

1-element Array{Int64,1}:
 -36

In [34]:
N = 100000
@time method1(N)
@time method2(N)
@time method3(N)

 18.783738 seconds (5.89 M allocations: 37.447 GB, 12.79% gc time)
  0.804275 seconds (21 allocations: 2.001 MB, 0.23% gc time)
  0.780113 seconds (6 allocations: 781.484 KB)


100000-element Array{Int64,1}:
 -18
  56
  20
  24
 -56
 -20
  42
 -22
  16
 -10
  16
 -32
  18
   ⋮
 -10
  -8
  -4
 -34
  36
 -52
  44
  -6
  30
  52
  16
 -28

The correct way to profile is with `@benchmark` from `BenchmarkTools.jl`

In [37]:
using BenchmarkTools

N = 10000
display(@benchmark method1(N))
display(@benchmark method2(N))
display(@benchmark method3(N))

BenchmarkTools.Trial: 
  memory estimate:  401.29 mb
  allocs estimate:  581300
  --------------
  minimum time:     380.563 ms (12.89% GC)
  median time:      423.159 ms (12.63% GC)
  mean time:        455.725 ms (15.49% GC)
  maximum time:     669.648 ms (29.66% GC)
  --------------
  samples:          11
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

BenchmarkTools.Trial: 
  memory estimate:  256.70 kb
  allocs estimate:  14
  --------------
  minimum time:     74.918 ms (0.00% GC)
  median time:      79.899 ms (0.00% GC)
  mean time:        79.730 ms (0.00% GC)
  maximum time:     84.844 ms (0.00% GC)
  --------------
  samples:          63
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

BenchmarkTools.Trial: 
  memory estimate:  78.20 kb
  allocs estimate:  2
  --------------
  minimum time:     73.927 ms (0.00% GC)
  median time:      78.467 ms (0.00% GC)
  mean time:        79.268 ms (0.00% GC)
  maximum time:     95.222 ms (0.00% GC)
  --------------
  samples:          64
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

## Collecting the random walk data

In [39]:
numsteps   = 1000
numwalkers = 10000

@time data = [walk(numsteps) for i in 1:numwalkers]  # final positions

  0.101981 seconds (12.15 k allocations: 621.639 KB)


10000-element Array{Int64,1}:
 -10
   8
 -52
 -36
   2
  -4
  34
   2
  32
 -14
 -38
  36
 -22
   ⋮
 -34
 -40
  28
 -10
  38
  12
  -2
  -2
  42
  18
  50
  18

### Let's do statistics

A population of simple random walks should have mean $0$ and variance equal to the total time. Let's check it:

In [44]:
mean(data)  # mean should be 0 for simple (unbiased) random walks

-0.2056

In [46]:
var(data), numsteps  # variance should be exactly the number of steps

(1020.6405926992701,1000)

In [49]:
≈(var(data), numsteps, rtol=3e-2)  # relative tolerance

true

We can plot the histogram:

In [50]:
using Plots; gr()

histogram(data, nbins=100)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -100 
 
 
 0 
 
 
 100 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

Again, however, the high number of allocations is suspect -- in Julia, this is usually a warning that there is a "type instability". We again try wrapping it in a function, even though it seems so simple:

In [40]:
function run_walks(numwalkers, numsteps)

    data = [walk(numsteps) for i in 1:numwalkers]

    return data
end

run_walks (generic function with 1 method)

In [42]:
numsteps   = 1000
numwalkers = 10000

data = run_walks(1, 1)  # compile the function

# now profile it:
@time data = run_walks(numwalkers, numsteps);  

  0.077005 seconds (7 allocations: 78.375 KB)


Summary:
- Simple simulation
- Put everything in a function
- Collect data using comprehensions or `push!`
- Use `@time` (on the second run) or `@benchmark` to profile

# DistributedArrays 

Jump to [notebook 1a.](1a. Basics of distributed arrays.ipynb) for the basics of distributed arrays in Julia.

# Basic parallelism

In [ ]:
addprocs(2)

In [ ]:
@everywhere using DistributedArrays

In [ ]:
@everywhere function walk(numsteps)
    pos = 0

    for j in 1:numsteps
        
        if rand(Bool)  # NB
            step = -1
        else
            step = +1
        end
        
        pos += step # ifelse(rand() < 0.5, -1, +1)
    end
    
    return pos
end

In [ ]:
walkers = distribute(1:numwalkers);

In [ ]:
walkers.indexes

In [ ]:
@everywhere begin
    numsteps   = 10000
    numwalkers = 100000 
end

walkers = distribute(1:numwalkers);

@time positions = map( _ -> walk(numsteps), walkers)

In [ ]:
positions

In [ ]:
mean(positions)

In [ ]:
var(positions)

In [ ]:
squared_positions = map(x->x^2, positions);

In [ ]:
mean(squared_positions) ≈ var(positions)

# Another example: random matrices

In [ ]:
using Plots; gr()

In [ ]:
addprocs(4)

In [ ]:
@everywhere begin
    using DistributedArrays
    using StatsBase
    using Plots
end

In [ ]:
@everywhere function stochastic(β = 2, n = 200)
    h = n ^ -(1/3)
    x = 0:h:10
    N = length(x)
    d = (-2 / h^2 .- x) + 2*sqrt(h*β) * randn(N) # diagonal
    e = ones(N - 1) / h^2                     # subdiagonal
  
    eigvals(SymTridiagonal(d, e))[N]        # smallest negative eigenvalue
end

Serial version:

In [ ]:
println("Serial version")

t = 10000
p = plot()
for β = [1,2,4,10,20]
    
    z = fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end
p

A related parallel construct: `@parallel`. This does a "reduce" operation.

In [ ]:
println("@parallel version")

@everywhere t = 10000

p = plot()

for β = [1,2,4,10,20]
    
    z = @parallel (+) for p = 1:nprocs()
        fit(Histogram, [stochastic(β) for i = 1:t], -4:0.01:1).weights
    end
    
    plot!(midpoints(-4:0.01:1), z / sum(z) / 0.01)
end

p

In [ ]:
function dhist(x; closed=:left, nbins=10)
    
    hist_parts = DArray(p->fit(Histogram, localpart(x), closed=closed, nbins=nbins).weights, (nbins*length(x.pids),))
    
    reduce(+, map(pid -> @fetchfrom(pid, localpart(hist_parts)), hist_parts.pids))
      
end

In [ ]:
a = randn(10000)
d = distribute(a)

dhist(d)